# **Case Study: Condition Monitoring of a Complex Hydraulic System**
The system under study consist of a primary working circuit (**a** in figure below) and a secundary circuit for cooling and filtration (**b** in figure below). This system was developed as a test rig allowing to change the state or condition of various components. [Paper available here](https://ieeexplore.ieee.org/document/7151267)

![picture](https://drive.google.com/uc?export=view&id=1Phrt8RMtkQ513iRafEY9rcsKvmszN9wh)

The given test system is equipped with **several sensors measuring process values** such as pressure (PS1-PS6), flow (FS1, FS2), temperature (TS1-TS5), electrical power (EPS1) and vibration (VS1). In addition, sensors for particle contamination (CS and MCS, COPS) and oil parameter monitoring (COPS) are integrated. 


The system cyclically repeats constant load cycles (duration 60 seconds) and measures process values **while the condition of four hydraulic components (cooler, valve, pump and accumulator) is quantitatively varied**. The dataset is available in the following [link](https://archive-beta.ics.uci.edu/dataset/447/condition+monitoring+of+hydraulic+systems).

# **Data Inspection**

The data has been collected from **17 sensors** (6 of pressure, 1 motor power, 2 volume flow, 4 temperature, 1 vibration, 1 cooling efficiency, 1 cooling power and 1 efficiency factor) over **2205 cycles** of 60 sec each (1.5 days). The sensors' sampling rate in different depending on the dynamics of the underlying physical values. The sensors involved with their respective unit of measure and samplig rate are reported in the Table below.


| Sensor | Physical quantity          | Unit  | Sampling rate |
|:--------|:---------------------------|:-------|:---------------|
| PS1    | Pressure                  | bar   | 100 Hz        |
| PS2    | Pressure                  | bar   | 100 Hz        |
| PS3    | Pressure                  | bar   | 100 Hz        |
| PS4    | Pressure                  | bar   | 100 Hz        |
| PS5    | Pressure                  | bar   | 100 Hz        |
| PS6    | Pressure                  | bar   | 100 Hz        |
| EPS1   | Motor power               | W     | 100 Hz        |
| FS1    | Volume flow               | l/min | 10 Hz         |
| FS2    | Volume flow               | l/min | 10 Hz         |
| TS1    | Temperature               | °C    | 1 Hz          |
| TS2    | Temperature               | °C    | 1 Hz          |
| TS3    | Temperature               | °C    | 1 Hz          |
| TS4    | Temperature               | °C    | 1 Hz          |
| VS1    | Vibration                 | mm/s  | 1 Hz          |
| CE     | Cooling efficiency (virtual) | %    | 1 Hz          |
| CP     | Cooling power (virtual)   | kW    | 1 Hz          |
| SE     | Efficiency factor         | %     | 1 Hz          |

The raw process sensors output is provided in separate dedicated files (one file per sensor). Each file is structured as a matrix (tab-delimited) with the rows representing the cycles and the columns the data samples within a cycle.

In order to simulate different scenarios, from normal functioning to faults of different severities, especific parameters of some components are configurable. In the following table are listed the configurable components and the respective controllable parameter.

| |Component  |Condition              |Control Parameter    |
|:-:|:-----------|:-----------------------|:---------------------|
|1|Cooler (C1)| Cooling power decrease| Fan duty cycle of C1|
|2|Valve (V10)| Switching characteristic degradation| Control current of V10|
|3|Pump (MP1)| Internal leakage|Switchable bypass orifices (V9)|
|4|Accumulators (A1-A4)| Gas leakage| Accumulators A1-A4 with different precharge pressure|

The condition of each component cycle-wise is provided in the file ```profile.txt```. ```profile.txt``` contains a matrix whose rows represent the cycles (2205), all columns but number 5 are the condition of each component and the last column is a flag indicating if stady state is reached or not. The columns order and entries are read in the following way:

1. **Cooler condition** (%):
	* 3: close to total failure
	* 20: reduced effifiency
	* 100: full efficiency

2. **Valve condition** (%):
	* 100: optimal switching behavior
	* 90: small lag
	* 80: severe lag
	* 73: close to total failure

3. **Internal pump leakage**:
	* 0: no leakage
	* 1: weak leakage
	* 2: severe leakage

4. **Hydraulic accumulator** (bar):
	* 130: optimal pressure
	* 115: slightly reduced pressure
	* 100: severely reduced pressure
	* 90: close to total failure

5. **Stable flag**:
	* 0: conditions were stable
	* 1: static conditions might not have been reached yet



<!--- 

The number of instances is 2205, while the number of attrbiutes is 3680 (8x60 (1 Hz) + 2x600 (10 Hz) + 7x6000 (100 Hz)). The attributes are the sensor data(all numeric and continuous) from measurements taken at the same point in time, respectively, of a hydraulic test rig's working cycle.) 

The sensors were read with different sampling rates, leading to different numbers of attributes per sensor despite they were all exposed to the same working cycle.
   
   1. Pressure sensors (PS1-6): 100 Hz, 6000 attributes per sensor (6 sensors)
   2. Motor power sensor (EPS1): 100 Hz, 6000 attributes per sensor (1 sensor)
   3. Volume flow sensors (FS1/2): 10 Hz, 600 attributes per sensor (2 sensors)
   4. Temperature sensors (TS1-4): 1 Hz, 60 attributes per sensor (4 sensors)
   5. Vibration sensor (VS1): 1 Hz, 60 attributes per sensor (1 sensor)
   6. Efficiency factor (SE): 1 Hz, 60 attributes per sensor (1 sensor)
   7. Virtual cooling efficiency sensor (CE): 1 Hz, 60 attributes per sensor (1 sensor)
   8. Virtual cooling power sensor (CP): 1 Hz, 60 attributes per sensor (1 sensor)

Note that if an attribute value is missing it has None value.

-->

# Data Loading
Due to the different sampling rates, the data is stored in groups by sampling period

EXPLAIN DATA SUBSAMPLING

In [21]:
import numpy as np
import matplotlib.pyplot as plt
import os
import csv
import graphviz as gr
import ges
def load_data(dir_array, dir_add, idx_arr=[]):
    i = 0
    for file in dir_array:
        temp = np.loadtxt(dir_add+file, delimiter='\t', skiprows=0, dtype=float)
        temp = temp[idx_arr,:]
        temp = temp.flatten()
        if i ==0:
            X = np.array([], dtype=np.int64).reshape(len(temp),0)
        X = np.hstack([X, temp.reshape(-1,1)])
        i=i+1
    return X

def save_adjMat_score(file_name, estimate, score):
    # open the file in the write mode
    f = open('Results/adjMat_' + file_name, 'w')
    # create the csv writer
    writer = csv.writer(f)
    for i in range(estimate.shape[0]):
        # write a row to the csv file
        writer.writerow(estimate[i,:])
    # close the file
    f.close()
    print('File saved: ', 'adjMat_' + file_name)
    
    f = open('Results/score_'+file_name+'.txt', 'w')
    # create the csv writer
    f.write(str(score))
    # close the file
    f.close()
    print('File saved: ', 'score_' + file_name)
    
# def create_graph(adjMat):
#     f = gr.Digraph(filename = 'graph1.gv')
#     for name in nodes:
#         f.node(name, name)

#     # Specify edges
#     for row in range(17):
#         for col in range(17):
#             if adjMat[row,col] == 1:
#                 f.edge(nodes[row],nodes[col])
#     return f
def create_graph(adjMat, nodes, labels):
    f = gr.Digraph(filename = 'graph1.gv')
    for name,label in zip(nodes,labels):
        f.node(name, label)

    # Specify edges
    for row in range(adjMat.shape[0]):
        for col in range(adjMat.shape[1]):
            if adjMat[row,col] == 1:
                f.edge(nodes[row],nodes[col])
    return f

nodes_full = ["CE","CP","SE","TS1","TS2","TS3","TS4","VS1","FS1","FS2","EPS1","PS1","PS2","PS3","PS4","PS5","PS6"]
# label =["CE","CP","SE","TS1","TS2","TS3","TS4","VS1","FS1","FS2","EPS1","PS1","PS2","PS3","PS4","PS5","PS6"]

In [22]:
# Removing non stable cycles
profile = np.loadtxt('data description/profile.txt', delimiter='\t', skiprows=0, dtype=float)
idx_stable = np.where(profile[:,4] == 0)
idx_stable = np.asarray(idx_stable)[0]
# Loading data sampled at 1Hz
print('----------------------------------------')
print('1Hz files')
print('----------------------------------------')
arr = os.listdir('data/1Hz')
X_1Hz = load_data(arr,'data/1Hz/', idx_arr=idx_stable) 
print(arr)
print('Data loaded. Matrix shape:', X_1Hz.shape)

# Loading data sampled at 1Hz
print('----------------------------------------')
print('10Hz files')
print('----------------------------------------')
arr = os.listdir('data/10Hz')
X_10Hz = load_data(arr,'data/10Hz/', idx_arr=idx_stable)
print(arr)
print('Data loaded. Matrix shape:', X_10Hz.shape)
# Data Sub sampling 
X_10Hz_sub=X_10Hz[0:-1:10,:]
print('Data sub sampled. Matrix shape:', X_10Hz_sub.shape)


# Loading data sampled at 1Hz
print('----------------------------------------')
print('100Hz files')
print('----------------------------------------')
arr = os.listdir('data/100Hz')
X_100Hz = load_data(arr,'data/100Hz/', idx_arr=idx_stable)
print(arr)
print('Data loaded. Matrix shape:', X_100Hz.shape)
# Data Sub sampling 
X_100Hz_sub=X_100Hz[0:-1:100,:]
print('Data sub sampled. Matrix shape:', X_100Hz_sub.shape)

# Data matrixes concatenation
print('----------------------------------------')
print('Whole data matrix created!!')
print('----------------------------------------')
data = np.concatenate((X_1Hz,X_10Hz_sub,X_100Hz_sub),1)
print(data.shape)

----------------------------------------
1Hz files
----------------------------------------
['CE.txt', 'CP.txt', 'SE.txt', 'TS1.txt', 'TS2.txt', 'TS3.txt', 'TS4.txt', 'VS1.txt']
Data loaded. Matrix shape: (86940, 8)
----------------------------------------
10Hz files
----------------------------------------
['FS1.txt', 'FS2.txt']
Data loaded. Matrix shape: (869400, 2)
Data sub sampled. Matrix shape: (86940, 2)
----------------------------------------
100Hz files
----------------------------------------
['EPS1.txt', 'PS1.txt', 'PS2.txt', 'PS3.txt', 'PS4.txt', 'PS5.txt', 'PS6.txt']
Data loaded. Matrix shape: (8694000, 7)
Data sub sampled. Matrix shape: (86940, 7)
----------------------------------------
Whole data matrix created!!
----------------------------------------
(86940, 17)


Figure below shows the measured values of three different sensors of the system during the whole measuring period (all 2205 cycles) 

SUBPLOT WITH ALL SENSORS

In [2]:
# Temperature (1 Hz)
t = np.linspace(0, 60*5, num=60*5)
plt.figure(figsize=(12,6))
plt.plot(t, X_1Hz[:60*5,3])
# plt.plot(t1, X_1Hz[0:-1:100,3])
plt.xlabel('Time [s]')
plt.ylabel('Temperature [°C]')
plt.title('TS1 sensor')
plt.ylim(30,60)
plt.show
print(X_1Hz[:,3].shape)

# Volume Flow (10 Hz)
t = np.linspace(0, 60*5, num=600*5)
t_sub = np.linspace(0, 60*5, num=60*5)
plt.figure(figsize=(12,6))
plt.plot(t, X_10Hz[:600*5,0])
plt.plot(t_sub, X_10Hz_sub[:60*5,0])
plt.xlabel('Time [s]')
plt.ylabel('Volume flow [l/min]')
plt.title('FS1 sensor')
# plt.ylim(30,60)
plt.show
# print(X_10Hz[0:-1:10,0].shape)

# Volume Flow (100 Hz)
t = np.linspace(0, 60*5, num=6000*5)
t_sub = np.linspace(0, 60*5, num=60*5)
plt.figure(figsize=(12,6))
plt.plot(t, X_100Hz[:6000*5,1])
plt.plot(t_sub, X_100Hz_sub[0:60*5,1])
plt.xlabel('Time [s]')
plt.ylabel('Pressure [bar]')
plt.title('PS1 sensor')
# plt.ylim(30,60)
plt.show
print(X_100Hz[0:-1:100,1].shape)

NameError: name 'X_1Hz' is not defined

<Figure size 1200x600 with 0 Axes>

In [ ]:
### This is a way to increment the number of samples within data
# import os
# # Data of 1Hz
# arr = os.listdir('data/1Hz')

# temp = np.loadtxt('data/1Hz/'+arr[0], delimiter='\t', skiprows=0, dtype=float)
# temp = temp.flatten()
# temp_ = np.asarray([])
# for ii in range(len(temp)):
#     for jj in range(100):
#         temp_ = np.append(temp_, temp[ii])

## Score based method: Greedy Equivalent Search 

### Whole data analysis

In [4]:
estimate,score = ges.fit_bic(data)

In [5]:
print(estimate,score)
save_adjMat_score('whole_data_sub',estimate,score)

[[0 0 0 0 0 0 0 1 0 0 0 0 1 1 1 0 0]
 [1 0 0 1 1 1 1 0 1 1 1 0 1 1 0 1 0]
 [1 0 0 0 0 1 1 1 0 1 0 0 1 1 1 1 0]
 [1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0]
 [1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 0 0 0 0 0 0 1 0 0 0 0 1 1 1 0 0]
 [1 0 0 0 0 1 0 0 0 1 0 0 1 1 1 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0]
 [1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 0]
 [1 0 0 0 0 1 0 1 0 0 0 0 1 1 0 0 0]
 [1 0 1 0 0 1 1 1 0 1 0 0 1 1 1 0 0]
 [1 0 1 1 1 0 1 1 1 1 1 0 1 1 1 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 1 1 0 0 1 0 0 1 1 1 0 1]
 [1 0 0 0 0 0 0 1 0 1 0 0 1 0 1 0 0]] 296062.73713914165
File saved:  adjMat_whole_data_sub
File saved:  score_whole_data_sub


In [4]:
f = create_graph(estimate, nodes_full, nodes_full)
f.render("Results/Graph_whole_data_sub", format="pdf")
f.render("Results/Graph_whole_data_sub", format="png")

NameError: name 'estimate' is not defined

### Severe leakage pump analysis

In [23]:
profile = np.loadtxt('data description/profile.txt', delimiter='\t', skiprows=0, dtype=float)
idx_MP1_2 = np.where((profile[:,4] == 0) & (profile[:,2] == 2))
idx_MP1_2 = np.asarray(idx_MP1_2)[0]

print('----------------------------------------')
print('1Hz files')
print('----------------------------------------')
arr = os.listdir('data/1Hz')
MP1_2_1Hz = load_data(arr,'data/1Hz/', idx_arr=idx_MP1_2) 
print(arr)
print('Data loaded. Matrix shape:', MP1_2_1Hz.shape)

# Loading data sampled at 1Hz
print('----------------------------------------')
print('10Hz files')
print('----------------------------------------')
arr = os.listdir('data/10Hz')
MP1_2_10Hz = load_data(arr,'data/10Hz/',idx_arr=idx_MP1_2)
print(arr)
print('Data loaded. Matrix shape:', MP1_2_10Hz.shape)
# Data Sub sampling 
MP1_2_10Hz_sub=MP1_2_10Hz[0:-1:10,:]
print('Data sub sampled. Matrix shape:', MP1_2_10Hz_sub.shape)


# Loading data sampled at 1Hz
print('----------------------------------------')
print('100Hz files')
print('----------------------------------------')
arr = os.listdir('data/100Hz')
MP1_2_100Hz = load_data(arr,'data/100Hz/', idx_arr=idx_MP1_2)
print(arr)
print('Data loaded. Matrix shape:', MP1_2_100Hz.shape)
# Data Sub sampling 
MP1_2_100Hz_sub=MP1_2_100Hz[0:-1:100,:]
print('Data sub sampled. Matrix shape:', MP1_2_100Hz_sub.shape)

# Data matrixes concatenation
print('----------------------------------------')
print('Whole data matrix created!!')
print('----------------------------------------')
MP1_2_data = np.concatenate((MP1_2_1Hz, MP1_2_10Hz_sub, MP1_2_100Hz_sub),1)
print(MP1_2_data.shape)

----------------------------------------
1Hz files
----------------------------------------
['CE.txt', 'CP.txt', 'SE.txt', 'TS1.txt', 'TS2.txt', 'TS3.txt', 'TS4.txt', 'VS1.txt']
Data loaded. Matrix shape: (28800, 8)
----------------------------------------
10Hz files
----------------------------------------
['FS1.txt', 'FS2.txt']
Data loaded. Matrix shape: (288000, 2)
Data sub sampled. Matrix shape: (28800, 2)
----------------------------------------
100Hz files
----------------------------------------
['EPS1.txt', 'PS1.txt', 'PS2.txt', 'PS3.txt', 'PS4.txt', 'PS5.txt', 'PS6.txt']
Data loaded. Matrix shape: (2880000, 7)
Data sub sampled. Matrix shape: (28800, 7)
----------------------------------------
Whole data matrix created!!
----------------------------------------
(28800, 17)


In [24]:
MP1_2_est, MP1_2_score = ges.fit_bic(MP1_2_data)

In [25]:
print(MP1_2_est,MP1_2_score)
save_adjMat_score('MP1_severe_fail',MP1_2_est,MP1_2_score)

f = create_graph(MP1_2_est,nodes_full,nodes_full)
f.render("Results/Graph_MP1_severe_fail", format="pdf")
f.render("Results/Graph_MP1_severe_fail", format="png")

[[0 1 1 0 0 0 0 0 1 0 0 1 1 1 1 1 1]
 [0 0 0 0 0 0 0 0 1 0 0 0 1 1 0 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 1]
 [1 0 1 0 1 1 1 1 1 0 1 1 1 1 0 1 0]
 [1 1 1 1 0 0 1 1 1 1 0 0 1 1 1 1 1]
 [1 1 1 1 0 0 1 1 1 1 1 0 1 1 1 1 0]
 [1 1 1 0 0 0 0 0 1 1 0 1 1 1 1 0 1]
 [1 1 1 0 0 0 1 0 1 1 1 1 1 1 1 1 1]
 [0 0 1 0 0 0 0 0 0 0 0 0 1 1 1 0 1]
 [1 1 1 0 0 0 0 0 1 0 0 1 1 0 1 1 1]
 [1 1 1 0 0 0 1 0 1 1 0 1 1 1 0 1 1]
 [0 0 1 0 0 0 0 0 1 0 0 0 1 1 1 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0]] 132238.01735495232
File saved:  adjMat_MP1_severe_fail
File saved:  score_MP1_severe_fail


'Results\\Graph_MP1_severe_fail.png'

### Severe leakage pump and V10 100% open analysis

In [12]:
profile = np.loadtxt('data description/profile.txt', delimiter='\t', skiprows=0, dtype=float)
idx_fail2 = np.where((profile[:,2] == 2) & (profile[:,1] == 100)& (profile[:,4] == 0))

idx_fail2 = np.asarray(idx_fail2)[0]

print('----------------------------------------')
print('1Hz files')
print('----------------------------------------')
arr = os.listdir('data/1Hz')
fail2_1Hz = load_data(arr,'data/1Hz/', idx_arr=idx_fail2) 
print(arr)
print('Data loaded. Matrix shape:', fail2_1Hz.shape)

# Loading data sampled at 1Hz
print('----------------------------------------')
print('10Hz files')
print('----------------------------------------')
arr = os.listdir('data/10Hz')
fail2_10Hz = load_data(arr,'data/10Hz/', idx_arr=idx_fail2)
print(arr)
print('Data loaded. Matrix shape:', fail2_10Hz.shape)
# Data Sub sampling 
fail2_10Hz_sub=fail2_10Hz[0:-1:10,:]
print('Data sub sampled. Matrix shape:', fail2_10Hz_sub.shape)


# Loading data sampled at 1Hz
print('----------------------------------------')
print('100Hz files')
print('----------------------------------------')
arr = os.listdir('data/100Hz')
fail2_100Hz = load_data(arr,'data/100Hz/', idx_arr=idx_fail2)
print(arr)
print('Data loaded. Matrix shape:', fail2_100Hz.shape)
# Data Sub sampling 
fail2_100Hz_sub=fail2_100Hz[0:-1:100,:]
print('Data sub sampled. Matrix shape:', fail2_100Hz_sub.shape)

# Data matrixes concatenation
print('----------------------------------------')
print('Whole data matrix created!!')
print('----------------------------------------')
fail2_data = np.concatenate((fail2_1Hz, fail2_10Hz_sub, fail2_100Hz_sub),1)
print(fail2_data.shape)

----------------------------------------
1Hz files
----------------------------------------
['CE.txt', 'CP.txt', 'SE.txt', 'TS1.txt', 'TS2.txt', 'TS3.txt', 'TS4.txt', 'VS1.txt']
Data loaded. Matrix shape: (7200, 8)
----------------------------------------
10Hz files
----------------------------------------
['FS1.txt', 'FS2.txt']
Data loaded. Matrix shape: (72000, 2)
Data sub sampled. Matrix shape: (7200, 2)
----------------------------------------
100Hz files
----------------------------------------
['EPS1.txt', 'PS1.txt', 'PS2.txt', 'PS3.txt', 'PS4.txt', 'PS5.txt', 'PS6.txt']
Data loaded. Matrix shape: (720000, 7)
Data sub sampled. Matrix shape: (7200, 7)
----------------------------------------
Whole data matrix created!!
----------------------------------------
(7200, 17)


In [ ]:
fail2_est, fail2_score = ges.fit_bic(fail2_data)

In [ ]:
print(fail2_est,fail2_score)
save_adjMat_score('fail2',fail2_est,fail2_score)

f = create_graph(fail2_est,nodes_full,nodes_full)
f.render("Results/Graph_fail2", format="pdf")
f.render("Results/Graph_fail2", format="png")

### V10 100% open analysis

In [ ]:
profile = np.loadtxt('data description/profile.txt', delimiter='\t', skiprows=0, dtype=float)
idx_fail3 = np.where(profile[:,1] == 100)

idx_fail3 = np.asarray(idx_fail3)[0]

print('----------------------------------------')
print('1Hz files')
print('----------------------------------------')
arr = os.listdir('data/1Hz')
fail3_1Hz = load_data(arr,'data/1Hz/', idx_arr=idx_fail3) 
print(arr)
print('Data loaded. Matrix shape:', fail3_1Hz.shape)

# Loading data sampled at 1Hz
print('----------------------------------------')
print('10Hz files')
print('----------------------------------------')
arr = os.listdir('data/10Hz')
fail3_10Hz = load_data(arr,'data/10Hz/', idx_arr=idx_fail3)
print(arr)
print('Data loaded. Matrix shape:', fail3_10Hz.shape)
# Data Sub sampling 
fail3_10Hz_sub=fail3_10Hz[0:-1:10,:]
print('Data sub sampled. Matrix shape:', fail3_10Hz_sub.shape)


# Loading data sampled at 1Hz
print('----------------------------------------')
print('100Hz files')
print('----------------------------------------')
arr = os.listdir('data/100Hz')
fail3_100Hz = load_data(arr,'data/100Hz/', idx_arr=idx_fail3)
print(arr)
print('Data loaded. Matrix shape:', fail3_100Hz.shape)
# Data Sub sampling 
fail3_100Hz_sub=fail3_100Hz[0:-1:100,:]
print('Data sub sampled. Matrix shape:', fail3_100Hz_sub.shape)

# Data matrixes concatenation
print('----------------------------------------')
print('Whole data matrix created!!')
print('----------------------------------------')
fail3_data = np.concatenate((fail3_1Hz, fail3_10Hz_sub, fail3_100Hz_sub),1)
print(fail3_data.shape)

In [ ]:
fail3_est, fail3_score = ges.fit_bic(fail3_data)

In [ ]:
print(fail3_est,fail3_score)
save_adjMat_score('fail3',fail3_est,fail3_score)

f = create_graph(fail3_est,nodes_full,nodes_full)
f.render("Results/Graph_fail3", format="pdf")
f.render("Results/Graph_fail3", format="png")

### C1 full efficiency and stability (stable flag = 0)

In [20]:
sensors = ["PS1","PS2","PS3","FS1","TS1","TS2","EPS1","VS1","CE","CP","SE"]

profile = np.loadtxt('data description/profile.txt', delimiter='\t', skiprows=0, dtype=float)
idx_fail4 = np.where((profile[:,0] == 100) & (profile[:,1] == 100) & (profile[:,2] == 2) & (profile[:,3] == 130) & (profile[:,4] == 0))
idx_fail4 = np.asarray(idx_fail4)[0]

print('----------------------------------------')
print('1Hz files')
print('----------------------------------------')
arr = os.listdir('data/1Hz')
arr1 = list()
for sensor in sensors:
    if sensor+'.txt' in arr:
        arr1.append(sensor+'.txt')
arr = arr1        
fail4_1Hz = load_data(arr,'data/1Hz/', idx_arr=idx_fail4) 
print(arr)
print('Data loaded. Matrix shape:', fail4_1Hz.shape)

# Loading data sampled at 1Hz
print('----------------------------------------')
print('10Hz files')
print('----------------------------------------')
arr = os.listdir('data/10Hz')
arr1 = list()
for sensor in sensors:
    if sensor+'.txt' in arr:
        arr1.append(sensor+'.txt')
arr = arr1   
print(arr)
fail4_10Hz = load_data(arr,'data/10Hz/', idx_arr=idx_fail4)
print('Data loaded. Matrix shape:', fail4_10Hz.shape)
# Data Sub sampling 
fail4_10Hz_sub=fail4_10Hz[0:-1:10,:]
print('Data sub sampled. Matrix shape:', fail4_10Hz_sub.shape)

# Loading data sampled at 100Hz
print('----------------------------------------')
print('100Hz files')
print('----------------------------------------')
arr = os.listdir('data/100Hz')
arr1 = list()
for sensor in sensors:
    if sensor+'.txt' in arr:
        arr1.append(sensor+'.txt')
arr = arr1
print(arr)
fail4_100Hz = load_data(arr,'data/100Hz/', idx_arr=idx_fail4)
print('Data loaded. Matrix shape:', fail4_100Hz.shape)
# Data Sub sampling 
fail4_100Hz_sub=fail4_100Hz[0:-1:100,:]
print('Data sub sampled. Matrix shape:', fail4_100Hz_sub.shape)

# Data matrixes concatenation
print('----------------------------------------')
print('Whole data matrix created!!')
print('----------------------------------------')
fail4_data = np.concatenate((fail4_1Hz, fail4_10Hz_sub, fail4_100Hz_sub),1)
print(fail4_data.shape)

----------------------------------------
1Hz files
----------------------------------------
['TS1.txt', 'TS2.txt', 'VS1.txt', 'CE.txt', 'CP.txt', 'SE.txt']
Data loaded. Matrix shape: (600, 6)
----------------------------------------
10Hz files
----------------------------------------
['FS1.txt']
Data loaded. Matrix shape: (6000, 1)
Data sub sampled. Matrix shape: (600, 1)
----------------------------------------
100Hz files
----------------------------------------
['PS1.txt', 'PS2.txt', 'PS3.txt', 'EPS1.txt']
Data loaded. Matrix shape: (60000, 4)
Data sub sampled. Matrix shape: (600, 4)
----------------------------------------
Whole data matrix created!!
----------------------------------------
(600, 11)


In [ ]:
fail4_est, fail4_score = ges.fit_bic(fail4_data)

In [ ]:
print(fail4_est,fail4_score)
save_adjMat_score('fail4',fail4_est,fail4_score)

f = create_graph(fail4_est,sensors,sensors)
f.render("Results/Graph_fail4", format="pdf")
f.render("Results/Graph_fail4", format="png")

##con questi dati GNN (600,8)

### C1 close total failure and stability (stable flag = 0)

In [ ]:
sensors = ["PS1.txt","PS2.txt","PS3.txt","FS1.txt","TS1.txt","TS2.txt","EPS1.txt","VS1.txt","TS5.txt","CE.txt","CP.txt","SE.txt"]

profile = np.loadtxt('data description/profile.txt', delimiter='\t', skiprows=0, dtype=float)
idx_fail5 = np.where((profile[:,0] == 3) & (profile[:,1] == 100) & (profile[:,2] == 2) & (profile[:,3] == 130) & (profile[:,4] == 0))
idx_fail5 = np.asarray(idx_fail5)[0]

print('----------------------------------------')
print('1Hz files')
print('----------------------------------------')
arr = os.listdir('data/1Hz')
arr1 = list()
for sensor in sensors:
    if sensor in arr:
        arr1.append(sensor)
arr = arr1        
fail5_1Hz = load_data(arr,'data/1Hz/',failure = True, idx_arr=idx_fail5) 
print(arr)
print('Data loaded. Matrix shape:', fail5_1Hz.shape)

# Loading data sampled at 1Hz
print('----------------------------------------')
print('10Hz files')
print('----------------------------------------')
arr = os.listdir('data/10Hz')
arr1 = list()
for sensor in sensors:
    if sensor in arr:
        arr1.append(sensor)
arr = arr1        
fail5_10Hz = load_data(arr,'data/10Hz/',failure = True, idx_arr=idx_fail5)
print(arr)
print('Data loaded. Matrix shape:', fail5_10Hz.shape)
# Data Sub sampling 
fail5_10Hz_sub=fail5_10Hz[0:-1:10,:]
print('Data sub sampled. Matrix shape:', fail5_10Hz_sub.shape)

# Loading data sampled at 100Hz
print('----------------------------------------')
print('100Hz files')
print('----------------------------------------')
arr = os.listdir('data/100Hz')
arr1 = list()
for sensor in sensors:
    if sensor in arr:
        arr1.append(sensor)
arr = arr1
print(arr)
fail5_100Hz = load_data(arr,'data/100Hz/',failure = True, idx_arr=idx_fail5)
print('Data loaded. Matrix shape:', fail5_100Hz.shape)
# Data Sub sampling 
fail5_100Hz_sub=fail5_100Hz[0:-1:100,:]
print('Data sub sampled. Matrix shape:', fail5_100Hz_sub.shape)

# Data matrixes concatenation
print('----------------------------------------')
print('Whole data matrix created!!')
print('----------------------------------------')
fail5_data = np.concatenate((fail5_1Hz, fail5_10Hz_sub, fail5_100Hz_sub),1)
print(fail5_data.shape)

In [ ]:
fail5_est, fail5_score = ges.fit_bic(fail5_data)

In [ ]:
print(fail5_est,fail5_score)
save_adjMat_score('fail5',fail5_est,fail5_score)

f = create_graph_reduced(fail5_est,sensors,sensors)
f.render("Results/Graph_fail5", format="pdf")
f.render("Results/Graph_fail5", format="png")

### C1 reduced efficiency and stability (stable flag = 0)

In [ ]:
sensors = ["PS1.txt","PS2.txt","PS3.txt","FS1.txt","TS1.txt","TS2.txt","EPS1.txt","VS1.txt","TS5.txt","CE.txt","CP.txt","SE.txt"]

profile = np.loadtxt('data description/profile.txt', delimiter='\t', skiprows=0, dtype=float)
idx_fail6 = np.where((profile[:,0] == 3) & (profile[:,1] == 100) & (profile[:,2] == 2) & (profile[:,3] == 130) & (profile[:,4] == 0))
idx_fail6 = np.asarray(idx_fail6)[0]

print('----------------------------------------')
print('1Hz files')
print('----------------------------------------')
arr = os.listdir('data/1Hz')
arr1 = list()
for sensor in sensors:
    if sensor in arr:
        arr1.append(sensor)
arr = arr1        
fail6_1Hz = load_data(arr,'data/1Hz/',failure = True, idx_arr=idx_fail6) 
print(arr)
print('Data loaded. Matrix shape:', fail6_1Hz.shape)

# Loading data sampled at 1Hz
print('----------------------------------------')
print('10Hz files')
print('----------------------------------------')
arr = os.listdir('data/10Hz')
arr1 = list()
for sensor in sensors:
    if sensor in arr:
        arr1.append(sensor)
arr = arr1        
fail6_10Hz = load_data(arr,'data/10Hz/',failure = True, idx_arr=idx_fail6)
print(arr)
print('Data loaded. Matrix shape:', fail6_10Hz.shape)
# Data Sub sampling 
fail6_10Hz_sub=fail6_10Hz[0:-1:10,:]
print('Data sub sampled. Matrix shape:', fail6_10Hz_sub.shape)

# Loading data sampled at 100Hz
print('----------------------------------------')
print('100Hz files')
print('----------------------------------------')
arr = os.listdir('data/100Hz')
arr1 = list()
for sensor in sensors:
    if sensor in arr:
        arr1.append(sensor)
arr = arr1
print(arr)
fail6_100Hz = load_data(arr,'data/100Hz/',failure = True, idx_arr=idx_fail6)
print('Data loaded. Matrix shape:', fail6_100Hz.shape)
# Data Sub sampling 
fail6_100Hz_sub=fail6_100Hz[0:-1:100,:]
print('Data sub sampled. Matrix shape:', fail6_100Hz_sub.shape)

# Data matrixes concatenation
print('----------------------------------------')
print('Whole data matrix created!!')
print('----------------------------------------')
fail6_data = np.concatenate((fail6_1Hz, fail6_10Hz_sub, fail6_100Hz_sub),1)
print(fail6_data.shape)

In [ ]:
fail6_est, fail6_score = ges.fit_bic(fail6_data)

In [ ]:
print(fail6_est,fail6_score)
save_adjMat_score('fail6',fail6_est,fail6_score)

f = create_graph_reduced(fail6_est,sensors,sensors)
f.render("Results/Graph_fail6", format="pdf")
f.render("Results/Graph_fail6", format="png")

In [ ]:
### GNN - Gcastle

In [ ]:
import os
os.environ['CASTLE_BACKEND'] = 'pytorch'
from castle.common import GraphDAG
from castle.algorithms import DAG_GNN
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

# rl learn
gnn4 = DAG_GNN(device_type='cpu')
gnn4.learn(fail4_data)
# plot dag
adj4 = gnn4.causal_matrix
GraphDAG(adj4)
sensors = ['PS1.txt','PS2.txt','PS3.txt','FS1.txt','TS1.txt','TS2.txt','EPS1.txt','VS1.txt','TS5.txt','CE.txt','CP.txt','SE.txt']

f = create_graph_reduced(adj4,sensors,sensors)
f.render("Results/Graph_fail4_GNN", format="pdf")
f.render("Results/Graph_fail4_GNN", format="png")

In [ ]:
# rl learn
gnn5 = DAG_GNN(device_type='cpu')
gnn5.learn(fail5_data)
# plot dag
adj5 = gnn5.causal_matrix
GraphDAG(adj5)
sensors = ['PS1.txt','PS2.txt','PS3.txt','FS1.txt','TS1.txt','TS2.txt','EPS1.txt','VS1.txt','TS5.txt','CE.txt','CP.txt','SE.txt']
f = create_graph_reduced(adj5,sensors,sensors)
f.render("Results/Graph_fail5_GNN", format="pdf")
f.render("Results/Graph_fail5_GNN", format="png")

In [ ]:
## rl learn
gnn6 = DAG_GNN(device_type='cpu')
gnn6.learn(fail6_data)
# plot dag
adj6 = gnn6.causal_matrix
GraphDAG(adj6)
sensors = ['PS1.txt','PS2.txt','PS3.txt','FS1.txt','TS1.txt','TS2.txt','EPS1.txt','VS1.txt','TS5.txt','CE.txt','CP.txt','SE.txt']
f = create_graph_reduced(adj6,sensors,sensors)
f.render("Results/Graph_fail6_GNN", format="pdf")
f.render("Results/Graph_fail6_GNN", format="png")

# **Bibliography:**

[1] https://archive-beta.ics.uci.edu/dataset/447/condition+monitoring+of+hydraulic+systems

[2] https://ieeexplore.ieee.org/document/7151267